In [2]:
import xfieldsdevlumi as xf
import xtrack as xt
import xpart as xp
import xobjects as xo
import numpy as np

context = xo.ContextCpu()
p0c = 6500e9
n_macroparticles = 1000
beta_x, beta_y = 19.17, 19.17
physemit_x, physemit_y = (2.946E-6 * xp.PROTON_MASS_EV) / p0c, (2.946E-6 * xp.PROTON_MASS_EV) / p0c
sigma_z, sigma_delta = 0.08, 1E-4
bunch_intensity = 0.7825E11

particles_b1 = xp.Particles(
    _context=context, p0c=p0c,
    x=np.sqrt(physemit_x * beta_x) * np.random.randn(n_macroparticles),
    px=np.sqrt(physemit_x / beta_x) * np.random.randn(n_macroparticles),
    y=np.sqrt(physemit_y * beta_y) * np.random.randn(n_macroparticles),
    py=np.sqrt(physemit_y / beta_y) * np.random.randn(n_macroparticles),
    zeta=sigma_z * np.random.randn(n_macroparticles),
    delta=sigma_delta * np.random.randn(n_macroparticles),
    weight=bunch_intensity / n_macroparticles
)

slicer = xf.TempSlicer(sigma_z=sigma_z, n_slices=30, mode='shatilov')

bbeam_b1 = xf.BeamBeamBiGaussian3D(
    _context=context,
    other_beam_q0=particles_b1.q0,  # Dummy self-interaction
    config_for_update=None,
    flag_luminosity=1,
    n_lumigrid_cells=1500,
    sig_lumigrid_cells=np.sqrt(physemit_x * beta_x),
    range_lumigrid_cells=24,
    update_lumigrid_sum=1
)

line = xt.Line(elements=[bbeam_b1])
line.build_tracker()

record = line.start_internal_logging_for_elements_of_type(xf.BeamBeamBiGaussian3D, 
    capacity={"lumitable": 10})

line.track(particles_b1, num_turns=3)
line.stop_internal_logging_for_elements_of_type(xf.BeamBeamBiGaussian3D)

print(record.lumitable.to_dict())  # Should show recorded values


Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


AssertionError: 